In [1]:
import numpy as np  # For numerical computations
import os  # Provides operating system functionalities
from sklearn.datasets import fetch_lfw_people  # For loading the LFW dataset
from sklearn.model_selection import train_test_split  # For splitting data into training and testing sets
from sklearn.metrics import classification_report, accuracy_score  # For evaluating model performance

In [2]:
# 1. Load and Preprocess Data
def load_and_preprocess_data():
    # Load LFW dataset, filtering people with at least 70 images and resizing them
    data = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
    X, y = data.images, data.target  # Extract image data and corresponding labels
    target_names = data.target_names  # Get the names of the classes

    # Normalize pixel values to the range [0, 1]
    X = X / 255.0

    return X, y, target_names

In [3]:
# 2. ResNet Implementation
class BasicResNetBlock:
    def __init__(self, input_dim, output_dim):
        # Initialize weights and biases for the block
        self.W = np.random.randn(input_dim, output_dim) * 0.1  # Small random weights
        self.b = np.zeros(output_dim)  # Bias initialized to zeros

    def forward(self, X):
        # Linear transformation: Z = X * W + b
        Z = np.dot(X, self.W) + self.b # this performs a linear transformation on the input X using the weights self.W and biases self.b.
        # ReLU activation: A = max(0, Z)
        A = np.maximum(0, Z) # this applies the ReLU activation function to the linear output 'Z'
        return A

class ResNet:
    def __init__(self, input_dim, output_dim, num_blocks):
        self.blocks = []  # List to hold all ResNet blocks
        current_dim = np.prod(input_dim)  # Calculates the flattened input size (e.g., height * width * channels)for compatability with fully connected layers

        # Create intermediate blocks
        for _ in range(num_blocks - 1):
            self.blocks.append(BasicResNetBlock(current_dim, current_dim))

        # Final block outputs the required dimension
        self.blocks.append(BasicResNetBlock(current_dim, output_dim))

    def forward(self, X):
        # Flatten input for compatibility with fully connected layers
        X_flat = X.reshape(X.shape[0], -1)
        output = X_flat
        # Pass input through each block sequentially
        for block in self.blocks:
            output = block.forward(output)
        return output

In [4]:
# 3. Decision Tree Implementation
class DecisionTreeNode:
    def __init__(self, depth=0, max_depth=10, label=None):
        self.depth = depth  # Depth of the node in the tree
        self.max_depth = max_depth  # Maximum depth allowed for the tree
        self.left = None  # Left child node
        self.right = None  # Right child node
        self.feature_index = None  # Index of the feature used for splitting
        self.threshold = None  # Threshold value for splitting
        self.label = label  # Class label if this is a leaf node

    def is_leaf(self):
        # Check if this node is a leaf
        return self.label is not None

class DecisionTree:
    def __init__(self, max_depth=10):
        self.max_depth = max_depth  # Maximum depth for the tree
        self.root = None  # Root node of the tree

    def fit(self, X, y):
        # Build the decision tree recursively
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape  # Get the number of samples and features
        if depth >= self.max_depth or len(set(y)) == 1:
            # Stop growing and create a leaf node if maximum depth is reached or only one class remains
            return DecisionTreeNode(depth=depth, label=max(set(y), key=list(y).count))

        # Find the best feature and threshold to split the data
        best_feature, best_threshold = self._best_split(X, y)
        if best_feature is None:
            # If no valid split is found, create a leaf node
            return DecisionTreeNode(depth=depth, label=max(set(y), key=list(y).count))

        # Split the data into left and right subsets
        left_indices = X[:, best_feature] <= best_threshold
        right_indices = X[:, best_feature] > best_threshold

        # Create a new node and recursively grow its left and right children
        node = DecisionTreeNode(depth=depth)
        node.feature_index = best_feature
        node.threshold = best_threshold
        node.left = self._grow_tree(X[left_indices], y[left_indices], depth + 1)
        node.right = self._grow_tree(X[right_indices], y[right_indices], depth + 1)
        return node

    def _best_split(self, X, y):
        num_samples, num_features = X.shape  # Get dimensions of the dataset
        if num_samples <= 1:
            return None, None

        best_gini = float('inf')  # Initialize best Gini impurity to a large value
        best_feature = None
        best_threshold = None

        # Iterate over all features
        for feature_index in range(num_features):
            thresholds = np.unique(X[:, feature_index])  # Unique values of the feature
            for threshold in thresholds:
                # Split the data
                left_indices = X[:, feature_index] <= threshold
                right_indices = X[:, feature_index] > threshold

                # Skip invalid splits
                if len(y[left_indices]) == 0 or len(y[right_indices]) == 0:
                    continue

                # Compute Gini impurity for the split
                gini = self._gini_impurity(y[left_indices], y[right_indices])

                # Update the best split if a lower Gini impurity is found
                if gini < best_gini:
                    best_gini = gini
                    best_feature = feature_index
                    best_threshold = threshold

        return best_feature, best_threshold

    def _gini_impurity(self, left_y, right_y):
        # Helper function to calculate Gini impurity
        def gini(y):
            m = len(y)
            if m == 0:
                return 0
            counts = np.bincount(y)  # Count occurrences of each class
            probabilities = counts / m  # Convert counts to probabilities
            return 1 - np.sum(probabilities**2)  # Gini formula

        m_left = len(left_y)
        m_right = len(right_y)
        m_total = m_left + m_right

        # Weighted Gini impurity of the split
        gini_left = gini(left_y)
        gini_right = gini(right_y)

        return (m_left / m_total) * gini_left + (m_right / m_total) * gini_right

    def predict(self, X):
        # Predict class labels for each sample in X
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        # Traverse the tree to make a prediction for a single sample
        if node.is_leaf():
            return node.label
        if x[node.feature_index] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)


In [5]:
# 4. Train and Evaluate
def main():
    # Load and preprocess data
    X, y, target_names = load_and_preprocess_data()

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Extract features using ResNet
    resnet = ResNet(input_dim=X_train.shape[1:], output_dim=128, num_blocks=3)
    X_train_features = resnet.forward(X_train)
    X_test_features = resnet.forward(X_test)

    # Train a decision tree on the extracted features
    dt = DecisionTree(max_depth=10)
    dt.fit(X_train_features, y_train)

    # Predict labels for the test set
    y_pred = dt.predict(X_test_features)

    # Calculate accuracy and display results
    accuracy = accuracy_score(y_test, y_pred)  # Compute accuracy
    print(f"Accuracy: {accuracy * 100:.2f}%")  # Print accuracy as a percentage
    print(classification_report(y_test, y_pred, target_names=target_names))  # Detailed report


In [6]:
if __name__ == "__main__":
    main() #intiating for main function

Accuracy: 39.15%
                   precision    recall  f1-score   support

     Ariel Sharon       0.04      0.09      0.06        11
     Colin Powell       0.38      0.40      0.39        47
  Donald Rumsfeld       0.15      0.09      0.11        22
    George W Bush       0.61      0.54      0.57       119
Gerhard Schroeder       0.25      0.37      0.30        19
      Hugo Chavez       0.12      0.08      0.10        13
       Tony Blair       0.23      0.26      0.24        27

         accuracy                           0.39       258
        macro avg       0.26      0.26      0.25       258
     weighted avg       0.41      0.39      0.40       258



In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import classification_report, accuracy_score

def build_cnn(input_shape, num_classes):
    # Build a CNN model
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),  # Convolution layer
        MaxPooling2D((2, 2)),  # Max-pooling layer
        Flatten(),  # Flatten the output of previous layer
        Dense(128, activation='relu'),  # Fully connected layer
        Dense(num_classes, activation='softmax')  # Output layer with softmax activation for classification
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])  # Compile the model
    return model


In [14]:
# Step 1: Load and preprocess data
X, y, target_names = load_and_preprocess_data()

# Add a channel dimension to the data
X = X[..., np.newaxis]  # Shape becomes (samples, height, width, 1)

# Step 2: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Build and train the CNN
cnn = build_cnn(input_shape=X_train.shape[1:], num_classes=len(target_names))
cnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Step 4: Evaluate the CNN
cnn_preds = cnn.predict(X_test)
cnn_preds_classes = np.argmax(cnn_preds, axis=1)

# Calculate and print accuracy
cnn_accuracy = accuracy_score(y_test, cnn_preds_classes)
print(f"CNN Accuracy: {cnn_accuracy * 100:.2f}%")

# Print classification report
print("CNN Classification Report:")
print(classification_report(y_test, cnn_preds_classes, target_names=target_names))


Epoch 1/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - accuracy: 0.3398 - loss: 1.8443 - val_accuracy: 0.4029 - val_loss: 1.6654
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - accuracy: 0.3817 - loss: 1.7420 - val_accuracy: 0.4029 - val_loss: 1.6658
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.4201 - loss: 1.6716 - val_accuracy: 0.4029 - val_loss: 1.6741
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.3868 - loss: 1.7258 - val_accuracy: 0.4029 - val_loss: 1.6636
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.3675 - loss: 1.7586 - val_accuracy: 0.4029 - val_loss: 1.6735
Epoch 6/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.3776 - loss: 1.7513 - val_accuracy: 0.4029 - val_loss: 1.6807
Epoch 7/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.4025 - loss: 1.7172 - val_accuracy: 0.4029 - val_loss: 1.6764
Epoch 8/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.4256 - loss: 1.6805 - val_accuracy: 0.4029 - v

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
